In [222]:
import io
import json
import os
import sys
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
from scipy import stats
import statistics as st

In [11]:
headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
}

In [3]:
r = requests.get("https://www.letrot.com/fr/courses/calendrier-resultats", headers=headers)

In [4]:
soup = bs(r.text, "html.parser")

In [30]:
courses[0].text[2:].replace(" ", "")

'VINCENNES'

In [31]:
courses[0].get("href")

'/stats/courses/programme/2022-11-03/7500'

In [13]:
def reunions(debut, fin):
    courses_dict = []
    url = f"https://www.letrot.com/fr/courses/calendrier-resultats?publish_up={debut}&publish_down={fin}"
    r = requests.get(url, headers=headers)
    soup = bs(r.text, "html.parser")
    courses_raw = soup.find_all("a", {"class": "racesHippodrome"})
    
    for c in courses_raw:
        date = c.get("href").split("/")[-2]
        courses_dict.append({"date": date, "idHippo": c.get("href").split("/")[-1], "Hippodrome": c.text[2:].replace(" ", ""), "lien": c.get("href")})
    
    return pd.DataFrame(courses_dict)

In [166]:
reu = reunions("02-11-2022", "03-11-2022")
reu.head()

,date,idHippo,Hippodrome,lien
0,2022-11-02,1302,MARSEILLE(AVIVAUX),/stats/courses/programme/2022-11-02/1302
1,2022-11-02,3103,TOULOUSE,/stats/courses/programme/2022-11-02/3103
2,2022-11-03,7500,VINCENNES,/stats/courses/programme/2022-11-03/7500


In [49]:
r = requests.get("https://www.letrot.com/stats/courses/programme/2018-11-01/5305/json", headers=headers)
r.json()

{'toReload': False,
 'nomHippodrome': 'LAVAL',
 'course': [{'heureCourse': '12:20',
   'discipline': 'Attelé',
   'numCourse': 1,
   'prix': 'PRIX DU CREDIT AGRICOLE',
   'allocation': '22 000',
   'distance': '2 875',
   'categorie': 'Course E',
   'typePiste': '',
   'conditionsEngagement': "Pour 6 ans, n'ayant pas gagné 135.000. - Recul de 25 m. à 76.000.<br />",
   'hasTracking': False,
   'hasVideoHeat': False,
   'statut': 16,
   'classement': '16 - 15 - 10 - 5 - 6',
   'linkPrix': 'https://www.letrot.com/stats/fiche-course/2018-11-01/5305/1/resultats/arrivee-definitive',
   'replay': '<a href="https://www.letrot.com/fr/replay-courses/2018-11-01/5305/1" class="btn" target="_blank"><svg width="8px" height="12px" viewBox="0 0 8 12" version="1.1" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"><g id="Page-1" stroke="none" stroke-width="1" fill="none" fill-rule="evenodd"><polygon id="triangle" fill="#ffffff" transform="translate(4.000000, 6.000000) rotat

In [136]:
def courses(reunions):
    courses_list = []
    
    for i, row in reunions.iterrows():
        url = f"https://www.letrot.com/{row['lien']}/json"
        r = requests.get(url, headers=headers)
        courses = r.json()
        for c in courses["course"]:
            if c["discipline"] == "Attelé":
                courses_list.append({"hippodrome": courses["nomHippodrome"],"date": row["date"], "idHippo": row["idHippo"],**c})
    return pd.DataFrame(courses_list)

In [167]:
courses_df = courses(reu)
courses_df.head()

,hippodrome,date,idHippo,heureCourse,discipline,numCourse,prix,allocation,distance,categorie,typePiste,conditionsEngagement,hasTracking,hasVideoHeat,statut,classement,linkPrix,replay
0,MARSEILLE (A VIVAUX),2022-11-02,1302,16:29,Attelé,1,PRIX DE LA TREILLE,24 000,2 650,Course F,,"Pour 6 à 9 ans inclus, n'ayant pas gagné 133.0...",False,True,16,13 - 10 - 9 - 15 - 7,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
1,MARSEILLE (A VIVAUX),2022-11-02,1302,17:04,Attelé,2,PRIX MAURICE UZEL,31 000,2 650,Course D,,"Pour 7 à 10 ans inclus, n'ayant pas gagné 305....",False,True,16,6 - 15 - 4 - 9 - 12,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
2,MARSEILLE (A VIVAUX),2022-11-02,1302,17:38,Attelé,3,PRIX DE LA BRIANDE,19 000,2 650,Course F,,"Pour 6 ans, n'ayant pas gagné 89.000. - Recul ...",False,False,16,6 - 3 - 2 - 8 - 11,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
3,MARSEILLE (A VIVAUX),2022-11-02,1302,18:13,Attelé,4,PRIX DE FONTANILLE,32 000,2 650,Course B,,Course B<br />Course Nationale<br />LeTROT Ope...,False,False,16,5 - 3 - 6 - 9 - 2,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
4,MARSEILLE (A VIVAUX),2022-11-02,1302,18:45,Attelé,5,PRIX DE VERNEGUES,28 000,2 650,Course B,,Course B<br />Course Nationale<br />LeTROT Ope...,False,False,16,7 - 14 - 9 - 11 - 12,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."


In [237]:
def partants(course):
    partants = []
    for i, course in course.iterrows():
        url = f"https://www.letrot.com/stats/fiche-course/{course['date']}/{course['idHippo']}/{course['numCourse']}/partants/tableau"
        r = requests.get(url, headers=headers)
        soup = bs(r.text, "html.parser")
        
        headers_table = soup.find("table", {"id": "result_table"}).find("thead").find("tr").find_all("th")
        table = soup.find("table", {"id": "result_table"}).find("tbody")
        rows = table.find_all("tr")
        
        for row in rows:
            num = row.select("td")[0].find("span", {"class": "bold"}).text
            col = row.select("td")
            if row.select("td") != "NP":
                cheval = {}
                cheval["nom"] = col[1].text
                cheval["date"] = course["date"]
                cheval["lien"] = col[1].find("a").get("href")
                
                cheval["fer"] = int(col[3].text) if col[3].text else 0
                cheval["firstTimeFer"] = True if col[3].find("div", {"class", "fer-first-time"}) else False
                cheval["sex"] = 0 if col[4].text == "M" else 1
                cheval["age"] = int(col[5].text)
                cheval["dist"] = int(col[6].text.replace(" ", "").replace("\n", ""))
                cheval["driver"] = col[7].find("a").get("href")
                cheval["trainer"] = col[8].find("a").get("href")
                
                if "Avis" in headers_table[9].text:
                    avis = col.pop(9)
                    col.insert(-1, avis)
                
                
                cheval["music"] = list(filter(lambda x: "a" in x, col[9].text.replace("D", "0").replace("Ret", "0").replace("T", "0").split()))
                cheval["music"] = list(map(lambda x: x[0], cheval["music"]))
                
                cheval["music"] = list(filter(lambda x: x.isnumeric(), cheval["music"]))
                
                cheval["music"] = list(map(int, cheval["music"]))
                cheval["nbArrivé"] = len(cheval["music"]) - cheval["music"].count("0")
                cheval["lastPerf"] = cheval["music"][0] if cheval["nbArrivé"] else 0
                
                arriveOnly = list(filter(None, cheval["music"]))
                
                cheval["meanPerf"] = np.mean(arriveOnly)
                cheval["medianPerf"] = np.median(arriveOnly)
                cheval["modePerf"] = st.mode(cheval["music"])
                
                try:
                    cheval["recordAbs"] = list(map(int, col[10].text.replace(col[10].span.text, "").replace("\'", '"').split('"')))
                    cheval["recordAbs"] = cheval["recordAbs"][0] * 10 * 60 + cheval["recordAbs"][1] * 10 + cheval["recordAbs"][2]
                except:
                    cheval["recordAbs"] = None

                cheval["gain"] = int(col[11].find("div", class_="gains").text.replace(" ", "")[:-1])
                
                partants.append(cheval)
        
    return pd.DataFrame(partants)

In [211]:
url = "https://www.letrot.com/stats/fiche-course/2022-11-02/1302/1/partants/tableau"
r = requests.get(url, headers=headers)
soup = bs(r.text, "html.parser")
table = soup.find("table", {"id": "result_table"})

In [216]:
"Avis" in table.find("thead").find("tr").find_all("th")[9].text

True

In [185]:
len(table.find("tbody").find_all("tr")[9].select("td"))

15

In [183]:
list(filter(lambda x: "a" in x, table.find("tbody").find_all("tr")[9].select("td")[9].text.replace("D", "0").split()))

[]

In [238]:
res = partants(courses_df)

In [239]:
res.tail(10)

,nom,date,lien,fer,firstTimeFer,sex,age,dist,driver,trainer,music,nbArrivé,lastPerf,meanPerf,medianPerf,modePerf,recordAbs,gain
273,HAPPY DJOB,2022-11-03,https://www.letrot.com/stats/fiche-cheval/happ...,3,False,0,5,2700,https://www.letrot.com/stats/fiche-homme/f-gue...,https://www.letrot.com/stats/fiche-homme/a-rip...,"[5, 3, 0, 2, 2, 1, 3, 4, 5, 0, 9, 2]",12,5,3.600000,3.0,2,719.0,100730
274,HYPER FAST,2022-11-03,https://www.letrot.com/stats/fiche-cheval/hype...,3,False,0,5,2700,https://www.letrot.com/stats/fiche-homme/f-lag...,https://www.letrot.com/stats/fiche-homme/f-sou...,"[7, 8, 9, 3, 7, 3, 2, 7, 7, 6, 2]",11,7,5.545455,7.0,7,711.0,102830
275,HOOPER DES CHASSES,2022-11-03,https://www.letrot.com/stats/fiche-cheval/hoop...,0,False,1,5,2700,https://www.letrot.com/stats/fiche-homme/r-bak...,https://www.letrot.com/stats/fiche-homme/p-van...,"[1, 1, 1, 5, 3, 1, 0, 9, 0, 7, 9, 0]",12,1,4.111111,3.0,1,727.0,102900
276,HERMES LUDOIS,2022-11-03,https://www.letrot.com/stats/fiche-cheval/herm...,3,False,1,5,2700,https://www.letrot.com/stats/fiche-homme/a-abr...,https://www.letrot.com/stats/fiche-homme/s-bou...,"[5, 2, 2, 2, 1, 0, 6, 0, 2, 3, 4, 1]",12,5,2.800000,2.0,2,730.0,102950
277,HOLERO LOVE,2022-11-03,https://www.letrot.com/stats/fiche-cheval/hole...,3,False,1,5,2700,https://www.letrot.com/stats/fiche-homme/d-tho...,https://www.letrot.com/stats/fiche-homme/s-gua...,"[4, 1, 1, 4, 0, 0, 2, 5, 2, 8, 1]",11,4,3.111111,2.0,1,723.0,105550
278,HELENCIO,2022-11-03,https://www.letrot.com/stats/fiche-cheval/hele...,3,False,0,5,2700,https://www.letrot.com/stats/fiche-homme/ch-ma...,https://www.letrot.com/stats/fiche-homme/a-van...,"[4, 0, 8, 3, 3, 1, 5, 4, 0, 3, 6, 3]",12,4,4.000000,3.5,3,712.0,105720
279,HIATUS,2022-11-03,https://www.letrot.com/stats/fiche-cheval/hiat...,3,False,1,5,2700,https://www.letrot.com/stats/fiche-homme/m-mot...,https://www.letrot.com/stats/fiche-homme/b-bou...,"[0, 0, 4, 4, 0, 2, 7, 0, 0, 0, 0]",11,0,4.250000,4.0,0,720.0,105830
280,HARRY CARISAIE,2022-11-03,https://www.letrot.com/stats/fiche-cheval/harr...,3,False,1,5,2700,https://www.letrot.com/stats/fiche-homme/b-roc...,https://www.letrot.com/stats/fiche-homme/je-th...,"[9, 1, 6, 1, 0, 6, 0, 9, 2, 2, 1]",11,9,4.111111,2.0,1,733.0,106320
281,HONNEUR DU CEBE,2022-11-03,https://www.letrot.com/stats/fiche-cheval/honn...,0,False,1,5,2700,https://www.letrot.com/stats/fiche-homme/f-poi...,https://www.letrot.com/stats/fiche-homme/f-leb...,"[0, 4, 5, 9, 6, 4, 0, 0, 2, 0, 9]",11,0,5.571429,5.0,0,722.0,107280
282,HAUTAIN,2022-11-03,https://www.letrot.com/stats/fiche-cheval/haut...,6,False,0,5,2700,https://www.letrot.com/stats/fiche-homme/y-leb...,https://www.letrot.com/stats/fiche-homme/v-leb...,"[0, 5, 2, 4, 3, 1, 0, 0, 0, 0, 7, 9]",12,0,4.428571,4.0,0,720.0,108530
